In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from openpyxl.styles import PatternFill
from openpyxl import Workbook
from openpyxl import load_workbook
from datetime import datetime, timedelta
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', None)


In [ ]:
drivers = pd.read_csv("f1db_csv/drivers.csv")
drivers = drivers[["driverId","number","code","forename","surname"]]
races = pd.read_csv("f1db_csv/races.csv")
races = races.drop(races.columns[[7,8,9,10,11,12,13,14,15,16,17]],axis = 1)
results = pd.read_csv("f1db_csv/results.csv")
qualifying = pd.read_csv("f1db_csv/qualifying.csv")
status = pd.read_csv("f1db_csv/status.csv")
constructors = pd.read_csv("f1db_csv/constructors.csv")
constructors = constructors[["constructorId","name"]]
driver_constructor = pd.read_csv("f1db_csv/driver_constructor.csv")
driver_constructor = driver_constructor.loc[(driver_constructor["year"]==2023)]

In [ ]:
driver_constructor =  driver_constructor[["driverId","constructorId"]].drop_duplicates().reset_index(drop=True)

In [ ]:
f1_team_colors = {
    'Mercedes': '#00D2BE',
    'Red Bull Racing': '#1E41FF',
    'McLaren': '#FF8700',
    'Ferrari': '#DC0000',
    'Alpine': '#0090FF',
    'AlphaTauri': '#2B4562',
    'Aston Martin': '#006F62',
    'Alfa Romeo Racing': '#900000',
    'Williams': '#005AFF',
    'Haas': '#F0D787'
}

In [ ]:
driver_constructor

,driverId,constructorId
0,1,131
1,4,117
2,807,210
3,815,9
4,817,213
5,822,51
6,825,210
7,830,9
8,832,6
9,839,214


In [ ]:
winners = results.loc[results["position"]=='1']

In [ ]:
winners = winners[["driverId","position"]]

In [ ]:
winners = pd.merge(winners,drivers,on='driverId')

In [ ]:
topwinners = winners.groupby(["driverId","forename","surname"]).count().sort_values(by='position',ascending=False).head(10)

In [ ]:
topwinners = topwinners[["position"]]

In [ ]:
topwinners.to_csv('csv_post_socialmedia/winners.csv')

### number of dnfs

In [ ]:
results = pd.merge(results,races[["raceId","year","round"]],on="raceId")
results = pd.merge(results,status,on='statusId')

In [ ]:
results2022 = results.loc[results["year"]==2022]
results2023 = results.loc[results["year"]==2023]

In [ ]:
retire2022 = results2022.loc[(results2022["statusId"]!=1)&(results2022["statusId"]<11)|(results2022["statusId"]>19)].sort_values(by='round')
retire2023 = results2023.loc[(results2023["statusId"]!=1)&(results2023["statusId"]<11)|(results2023["statusId"]>19)].sort_values(by='round')

In [ ]:
retire2022 = retire2022[["raceId","constructorId","driverId","number","positionText","statusId","year","round","status"]]
retire2023 = retire2023[["raceId","constructorId","driverId","number","positionText","statusId","year","round","status"]]

In [ ]:
retire2022 = retire2022.loc[retire2022["round"]<13]

In [ ]:
retire2023 = pd.merge(retire2023,constructors,on='constructorId')
retire2022 = pd.merge(retire2022,constructors,on='constructorId')

In [ ]:
# results2023 = results.loc[results["raceId"]>=1098].reset_index(drop=True)

In [ ]:
ret2023 = retire2023[["name","year","status"]].groupby(["name","year"]).agg({"status": "count"}).reset_index()

In [ ]:
ret2022 = retire2022[["name","year","status"]].groupby(["name","year"]).count().reset_index()

In [ ]:
total = pd.concat([ret2022,ret2023], axis=0)
total.to_csv('csv_post_socialmedia/dnfcomp.csv',index=False)

### pitstop

In [ ]:
pitstops = pd.read_csv("f1db_csv/pit_stops.csv")
pitstops = pd.merge(pitstops,races[["raceId","year","round"]],on='raceId')
pitstops.head()

,raceId,driverId,stop,lap,time,duration,milliseconds,year,round
0,841,153,1,1,17:05:23,26.898,26898,2011,1
1,841,30,1,1,17:05:52,25.021,25021,2011,1
2,841,17,1,11,17:20:48,23.426,23426,2011,1
3,841,4,1,12,17:22:34,23.251,23251,2011,1
4,841,13,1,13,17:24:10,23.842,23842,2011,1


In [ ]:
pitstops2023 = pitstops.loc[pitstops["year"]==2023]
pitstops2023 = pitstops2023.loc[pitstops2023["milliseconds"]<70000].sort_values(by=['round','duration'])

In [ ]:
pitstops2023 = pd.merge(pitstops2023,drivers,on='driverId')

In [ ]:
results.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,year,round,status
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1,2008,1,Finished
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1,2008,1,Finished
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1,2008,1,Finished
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1,2008,1,Finished
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1,2008,1,Finished


In [ ]:
pitstops2023 = pd.merge(pitstops2023,driver_constructor[["driverId","constructorId"]],on='driverId')


In [ ]:
pitstops2023 = pd.merge(pitstops2023,constructors,on='constructorId')

In [ ]:
pitstops2023.head()

,raceId,driverId,stop,lap,time,duration,milliseconds,year,round,number,code,forename,surname,constructorId,name
0,1098,832,2,31,18:55:11,24.227,24227,2023,1,55,SAI,Carlos,Sainz,6,Ferrari
1,1098,832,1,13,18:25:17,24.500,24500,2023,1,55,SAI,Carlos,Sainz,6,Ferrari
2,1099,832,1,15,20:27:49,20.760,20760,2023,2,55,SAI,Carlos,Sainz,6,Ferrari
3,1100,832,1,7,15:15:05,18.319,18319,2023,3,55,SAI,Carlos,Sainz,6,Ferrari
4,1101,832,1,11,15:24:33,21.970,21970,2023,4,55,SAI,Carlos,Sainz,6,Ferrari


In [ ]:
pitstops2023 = pitstops2023.sort_values(by='round')[["driverId","number","code","duration","milliseconds","year","round","constructorId","name"]]

In [ ]:
pitstops2023.head()

,driverId,number,code,duration,milliseconds,year,round,constructorId,name
0,832,55,SAI,24.227,24227,2023,1,6,Ferrari
220,4,14,ALO,25.800,25800,2023,1,117,Aston Martin
219,4,14,ALO,24.869,24869,2023,1,117,Aston Martin
201,840,18,STR,24.639,24639,2023,1,117,Aston Martin
180,858,2,SAR,25.670,25670,2023,1,3,Williams


In [ ]:
avgpitstops2023 = pitstops2023.groupby(["round","constructorId","name"])["milliseconds"].mean().reset_index()


In [ ]:
avgpitstops2023 = avgpitstops2023.sort_values(by=["round",'milliseconds'])

In [ ]:
avgpitstops2023["seconds"] = avgpitstops2023["milliseconds"]/1000

In [ ]:
avgpitstops2023.head(11)

,round,constructorId,name,milliseconds,seconds
2,1,6,Ferrari,24429.000000,24.429000
3,1,9,Red Bull,24638.500000,24.638500
1,1,3,Williams,25124.833333,25.124833
8,1,213,AlphaTauri,25141.800000,25.141800
5,1,117,Aston Martin,25189.250000,25.189250
6,1,131,Mercedes,25210.000000,25.210000
4,1,51,Alfa Romeo,26455.600000,26.455600
7,1,210,Haas F1 Team,27805.333333,27.805333
9,1,214,Alpine F1 Team,31074.333333,31.074333
0,1,1,McLaren,32555.666667,32.555667


In [ ]:
compavgpitstop = avgpitstops2023.loc[avgpitstops2023["round"] == 1]
compavgpitstop["gap"] = compavgpitstop["seconds"] - compavgpitstop["seconds"].min()

round_max = avgpitstops2023["round"].max()

for i in range(2, round_max + 1):
    df_round = avgpitstops2023.loc[avgpitstops2023["round"] == i]
    df_round["gap"] = df_round["seconds"] - df_round["seconds"].min()
    compavgpitstop = pd.concat([compavgpitstop, df_round])  # Pass a list of DataFrames to pd.concat



C:\Users\GTR\AppData\Local\Temp\ipykernel_22424\17162067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compavgpitstop["gap"] = compavgpitstop["seconds"] - compavgpitstop["seconds"].min()
C:\Users\GTR\AppData\Local\Temp\ipykernel_22424\17162067.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_round["gap"] = df_round["seconds"] - df_round["seconds"].min()
C:\Users\GTR\AppData\Local\Temp\ipykernel_22424\17162067.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [ ]:
compavgpitstop = compavgpitstop.groupby(["constructorId","name"])["gap"].mean().reset_index()
compavgpitstop["gaptime"] = compavgpitstop["gap"]
compavgpitstop["gaptime"] = compavgpitstop["gaptime"] - compavgpitstop["gap"].min()
compavgpitstop = compavgpitstop.sort_values("gaptime")

In [ ]:
compavgpitstop

,constructorId,name,gap,gaptime
3,9,Red Bull,0.251847,0.000000
2,6,Ferrari,0.668339,0.416492
6,131,Mercedes,0.775874,0.524027
8,213,AlphaTauri,0.987325,0.735477
5,117,Aston Martin,1.109056,0.857209
9,214,Alpine F1 Team,1.372090,1.120243
4,51,Alfa Romeo,1.831220,1.579373
1,3,Williams,2.041807,1.789960
0,1,McLaren,2.415912,2.164064
7,210,Haas F1 Team,3.423068,3.171221


In [ ]:
plt.figure(figsize=(12, 10))
x = compavgpitstop["name"]
y = compavgpitstop["gaptime"]


# Create the bar plot
plt.bar(x, y)

# Add labels and title
plt.xlabel('Drivers')
plt.ylabel('Top Speed (Km/h)')
plt.title('2023')
# plt.yticks(range(300, 400, 5))
# plt.ylim(300, 340)
# # Add text annotations on top of bars
# for i in range(len(x)):
#     plt.text(i, y.loc[i]+1, str(y.loc[i]), ha='center', va='bottom')

plt.grid(axis='y', alpha=0.2)
# Display the plot
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
x = ['A','B','C']
y = [10,15,20]


# Create the bar plot
plt.bar(x, y)

# Add labels and title
plt.xlabel('Drivers')
plt.ylabel('Top Speed (Km/h)')
plt.title('2023')
# plt.yticks(range(300, 400, 5))
# plt.ylim(300, 340)
# # Add text annotations on top of bars
# for i in range(len(x)):
#     plt.text(i, y.loc[i]+1, str(y.loc[i]), ha='center', va='bottom')

plt.grid(axis='y', alpha=0.2)
# Display the plot
plt.show()